In [1]:
from ris import pysqldb
from IPython.display import clear_output
from collections import defaultdict
from collections import defaultdict, namedtuple
import datetime 
import pandas as pd
import numpy as np
import os
from sqlalchemy import create_engine

clear_output()
timestamp = datetime.datetime.now().strftime('%Y-%m-%d_%H:%M')
ts = datetime.datetime.now().strftime('%Y-%m-%d')

print 'Notebook run: {}'.format(timestamp)
print os.getcwd()

Notebook run: 2020-01-10_19:58
E:\RIS\Staff Folders\Samuel\Requests\SH\Refrence_Marker_Crash_Request


In [2]:
sqldb = pysqldb.DbConnect(server='dot55sql01', database='forms', type='MS', user = 'arcgis', password = 'arcgis')

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-01-10 19:58:56
Query run in 54000 microseconds
	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [375]:
sqldb.connect()

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly

Database connection (MS) to forms on dot55sql01 - user: arcgis 
Connection established 2020-01-14 12:32:12


In [379]:
result = sqldb.dfquery("""    


with data as(
Select
core.OFF_STREET
,core.integration_id
,core.ACCIDENT_DT 
,row_number() over (partition by core.INTEGRATION_ID order by core.ACCIDENT_DT) as VEH
,veh.[ST_OF_REG]
,veh.VEHICLE_TYPE_CODE
,veh.[VEH_OCCUPANTS]
,cast(datediff(DD,veh.[DRIVER_DOB],getdate())/365.25 as int)  DRIVER_AGE
,veh.[DRIVER_SEX]
,veh.[VIOLATION]
,veh.[DIRECTION_OF_TRAVEL]
,veh.[PROPERTY_DAMAGED_DESC]
,veh.[CONTRIBUTING_FACTOR]
,veh.[PRE_ACDNT_ACTION]
FROM [FORMS].[dbo].[WC_ACCIDENT_F] core
JOIN FORMS.dbo.WC_ACCIDENT_VEHICLE_F veh
on core.integration_id = veh.accident_id
WHERE core.OFF_STREET in( '907AX5M21121', '907AX5M21122', '907AX5M21123', '907AX5M21124')
and YEAR(core.ACCIDENT_DT) between 2017 and 2019 
--order by core.OFF_STREET,core.ACCIDENT_DT
)

Select x.*
,d.integration_id int_id
,d.[ST_OF_REG]
,d.VEH
,d.[VEHICLE_TYPE_CODE]
,d.[VEH_OCCUPANTS]
,d.[DRIVER_AGE]
,d.[DRIVER_SEX]
,d.[VIOLATION]
,d.[DIRECTION_OF_TRAVEL]
,d.[PROPERTY_DAMAGED_DESC]
,d.[CONTRIBUTING_FACTOR]
,d.[PRE_ACDNT_ACTION]
FROM (
Select 
core.OFF_STREET
,core.integration_id, row_number() over (partition by core.OFF_STREET order by core.integration_id) as case_num
,core.ACCIDENT_DT
,core.SRC_CROSS_STREET
,core.SRC_ON_STREET
,core.SRC_DIST_FROM_X_STREET_MILES
,core.SRC_DIRECTION_FROM_X_STREET
,core.INJURED_CNT
,core.VEHICLE_INVOLVED_CNT
,core.ACCIDENT_TYPE
,core.TRAFFIC_CONTROL
,core.ACCIDENT_DIAGRAM
,core.ROADWAY_SURFACE_COND
,core.ROADWAY_CHARACTER
,core.LIGHT_CONDITIONS
,core.CONTRIBUTING_FACTOR1
,ACCIDENT_DESC
FROM [FORMS].[dbo].[WC_ACCIDENT_F] core
WHERE core.OFF_STREET in( '907AX5M21121', '907AX5M21122', '907AX5M21123', '907AX5M21124')
and YEAR(core.ACCIDENT_DT) between 2017 and 2019
--order by core.OFF_STREET
) x

JOIN (SELECT * FROM data) d
ON x.INTEGRATION_ID = d.integration_id
order by x.OFF_STREET, case_num

    """)

	Missing SQL Server Native Client 10.0                 datetime2 will not be interpreted correctly



In [49]:
result.OFF_STREET.unique()[1]

u'907AX5M21122'

In [50]:
 result.OFF_STREET[0]

u'907AX5M21121'

In [369]:
result.iloc[0:1,19::]

,VEH,ST_OF_REG,VEHICLE_TYPE_CODE,VEH_OCCUPANTS,DRIVER_DOB,DRIVER_SEX,VIOLATION,DIRECTION_OF_TRAVEL,PROPERTY_DAMAGED_DESC,CONTRIBUTING_FACTOR
0,1,NY,Motorcycle,1,1992-04-10,M,None,Northeast,None,Unspecified


In [71]:
result.loc[result.OFF_STREET==result.OFF_STREET[6]].case_num

119

### test 1

In [109]:
i = 0
j = 0
while i < range(len(result.OFF_STREET.unique())-1):
    while j<len(result):
        try:
            if result.OFF_STREET[j] == result.OFF_STREET.unique()[i]:
                print(result.iloc[j,0:9])
                case_amount = len((result.loc[result.OFF_STREET==result.OFF_STREET[j]].case_num))
                print(result.iloc[j:j+case_amount-1,17::])   
                print(j+case_amount-1)
                i+=1
                j+=1
            j+=1
            #print('i:',i,result.OFF_STREET[i] ,'j:',j, result.OFF_STREET[j])
        except:
            break
    break
    


    

OFF_STREET                                                           907AX5M21121
integration_id                                                         0309718105
case_num                                                                        1
ACCIDENT_DT                                                            2018-06-10
SRC_CROSS_STREET                                                             None
SRC_ON_STREET                   CROSS ISLAND PARKWAY                          ...
SRC_DIST_FROM_X_STREET_MILES                                                 None
SRC_DIRECTION_FROM_X_STREET                                                    N 
INJURED_CNT                                                                     1
Name: 0, dtype: object
  integration_id  VEH ST_OF_REG VEHICLE_TYPE_CODE VEH_OCCUPANTS  DRIVER_DOB  \
0     0309718105    1        NY        Motorcycle             1  1992-04-10   
1     0309718105    2      None             Sedan          None        None   
2 

[118 rows x 11 columns]
124
OFF_STREET                                                           907AX5M21123
integration_id                                                         0001819105
case_num                                                                        1
ACCIDENT_DT                                                            2019-01-01
SRC_CROSS_STREET                                                             None
SRC_ON_STREET                   CROSS ISLAND PARKWAY                          ...
SRC_DIST_FROM_X_STREET_MILES                                                 None
SRC_DIRECTION_FROM_X_STREET                                                    N 
INJURED_CNT                                                                     0
Name: 125, dtype: object
    integration_id  VEH ST_OF_REG                    VEHICLE_TYPE_CODE  \
125     0001819105    1        NY                                Sedan   
126     0001819105    2        NY  Station Wagon/Sport Utilit

170                  None                     Unspecified  
171
OFF_STREET                                                           907AX5M21124
integration_id                                                         0062817105
case_num                                                                        1
ACCIDENT_DT                                                            2017-02-06
SRC_CROSS_STREET                                                             None
SRC_ON_STREET                   CROSS ISLAND PARKWAY                          ...
SRC_DIST_FROM_X_STREET_MILES                                                 None
SRC_DIRECTION_FROM_X_STREET                                                    N 
INJURED_CNT                                                                     1
Name: 172, dtype: object
    integration_id  VEH ST_OF_REG                    VEHICLE_TYPE_CODE  \
172     0062817105    1        NY  Station Wagon/Sport Utility Vehicle   
173     0062817105    2  

###  test 2

In [212]:
writer = pd.ExcelWriter('test.xlsx',engine = 'xlsxwriter')

In [213]:
i = 0
j = 0
row = 0
while i < range(len(result.OFF_STREET.unique())-1):
    while j<len(result):
        try:
            if result.OFF_STREET[j] == result.OFF_STREET.unique()[i]:
                #print(result.iloc[j,0:9])
                result.iloc[j:j+1,0:16].to_excel(writer, sheet_name='Sheet1', startrow=row, index = False)
                case_amount = len((result.loc[result.OFF_STREET==result.OFF_STREET[j]].case_num))
                print (row)
                result.iloc[j:j+case_amount,17::].to_excel(writer, sheet_name='Sheet1', startcol=2, startrow=row+2, index = False)       
                i+=1
                j+=1
            j+=1
            row=j+3

            #print('i:',i,result.OFF_STREET[i] ,'j:',j, result.OFF_STREET[j])
        except:
            break
        
    break
    
writer.save()


0
9
128
175


### test 3 - Working Code

In [380]:
writer = pd.ExcelWriter('test2.xlsx',engine = 'xlsxwriter')

In [353]:
workbook  = writer.book
header_format = workbook.add_format({
    'bold': True,
    'fg_color': '#ffcccc',
    'border': 1})

In [237]:
result.loc[result.integration_id=='0309718105'].VEHICLE_INVOLVED_CNT

0    2
1    2
Name: VEHICLE_INVOLVED_CNT, dtype: int64

In [247]:
result.integration_id[0]

u'0309718105'

In [249]:
result.integration_id.unique()[0]

u'0309718105'

In [312]:
int(result.loc[result.integration_id == result.iloc[2].integration_id].VEHICLE_INVOLVED_CNT.head(1))

2

In [276]:
result.integration_id[12]

u'0033217105'

In [302]:
result.integration_id == result.iloc[2].integration_id

0      False
1      False
2       True
3       True
4      False
5      False
6      False
7      False
8      False
9      False
10     False
11     False
12     False
13     False
14     False
15     False
16     False
17     False
18     False
19     False
20     False
21     False
22     False
23     False
24     False
25     False
26     False
27     False
28     False
29     False
       ...  
162    False
163    False
164    False
165    False
166    False
167    False
168    False
169    False
170    False
171    False
172    False
173    False
174    False
175    False
176    False
177    False
178    False
179    False
180    False
181    False
182    False
183    False
184    False
185    False
186    False
187    False
188    False
189    False
190    False
191    False
Name: integration_id, Length: 192, dtype: bool

In [381]:
i = 0
j = 0
row = 0
count=0
while i < range(len(result.integration_id.unique())-1):
    while j<len(result):
        try:
            if result.integration_id[j] == result.integration_id.unique()[i]:
                #print(result.iloc[j:j+1,0:16])
                result.iloc[j:j+1,0:18].to_excel(writer, sheet_name='Sheet1', startrow=count, index = False)
                count+=2
                veh_cnt = veh_cnt = int(result.loc[result.integration_id == result.iloc[j].integration_id].VEHICLE_INVOLVED_CNT.head(1))
                result.iloc[j:j+veh_cnt,19::].to_excel(writer, sheet_name='Sheet1', startcol=2, startrow=count,index = False)       
                #print(result.iloc[j:j+veh_cnt,18::]) 
                i+=1
                #j+=1
                count+=veh_cnt+1
            #row+=veh_cnt+j+1
            j+=1
            #print('count:',count,'j:',j)
            #print('i:',i,result.integration_id[i] ,'j:',j,result.integration_id[j])
        except:
            break
        
    break
    
writer.save()


In [344]:
veh_cnt = int(result.loc[result.integration_id == result.iloc[83].integration_id].VEHICLE_INVOLVED_CNT.head(1))


In [345]:
veh_cnt

2

In [347]:
result.iloc[82:84,17::]

,int_id,VEH,ST_OF_REG,VEHICLE_TYPE_CODE,VEH_OCCUPANTS,DRIVER_DOB,DRIVER_SEX,VIOLATION,DIRECTION_OF_TRAVEL,PROPERTY_DAMAGED_DESC,CONTRIBUTING_FACTOR
82,0560617105,1,NY,Station Wagon/Sport Utility Vehicle,1,1964-11-04,F,None,North,None,Unspecified
83,0560617105,2,NY,Sedan,1,1958-09-29,M,None,North,None,Driver Inattention/Distraction
